In [1]:
import os
import sys
# Define the path to the project root directory (one level above `src`)
project_root_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH"

# Add the src directory to sys.path
sys.path.append(os.path.join(project_root_path))

# Verify that src is in sys.path
print(sys.path)
import os
import argparse
from src.utils.dataset_utils import create_directory
from src.visualizers.Open3dVisualizer import Open3DVisualizer
from src.utils.json_utils import load_cuboid_data
from src.utils.plotting_utils import load_mesh_data, load_pointcloud_data
from src.scripts.preprocessing_data.data_to_6views_utils import process_file
import numpy as np
from PIL import Image
import open3d as o3d

['/opt/anaconda3/envs/myenv/lib/python311.zip', '/opt/anaconda3/envs/myenv/lib/python3.11', '/opt/anaconda3/envs/myenv/lib/python3.11/lib-dynload', '', '/opt/anaconda3/envs/myenv/lib/python3.11/site-packages', '/Users/cmazzoleni/Documents/GitHub/Hierarchical-Localization/Hierarchical-Localization/Hierarchical-Localization', '/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH']


In [2]:
visualizer = Open3DVisualizer()
print(dir(visualizer))

mesh_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/raw/03001627/185bcb9bcec174c9492d9da2668ec34c/models/model_normalized.obj"
mesh = o3d.io.read_triangle_mesh(mesh_path)
visualizer.vis.add_geometry(mesh)

front_view_direction = [0, 0, -1]  # Front view direction (camera looking towards -z)
output_filename = "front_view_depth_mesh.png"
depth_o3d_filename = "front_view_depth_o3d_mesh.png"
    
visualizer.capture_single_view(front_view_direction, depth_o3d_filename, output_filename, save_depth=True)
    


[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'add_complete_cuboid', 'add_geometry', 'add_pointcloud', 'add_superquadric', 'capture_front_depth', 'capture_front_picture', 'capture_single_view', 'clear_scene', 'close', 'save_cube_map_screenshots', 'save_views', 'setup_render_options', 'show', 'vis', 'window_height', 'window_width']
Saved depth image to front_view_depth_mesh.png


In [7]:
import open3d as o3d
import numpy as np

def reconstruct_from_depth(depth_filename, width, height, fx, fy, cx, cy, output_pointcloud_filename):
    # Load the saved raw depth image (not normalized)
    depth_image = o3d.io.read_image(depth_filename)
    depth_np = np.asarray(depth_image)

    # Print the data type and shape of the depth image
    print(f"Depth image type: {depth_np.dtype}")
    print(f"Depth image shape: {depth_np.shape}")

    # Create an intrinsic matrix for the camera
    intrinsic = o3d.camera.PinholeCameraIntrinsic(width, height, fx, fy, cx, cy)

    # Convert the depth image to a point cloud
    pcd = o3d.geometry.PointCloud.create_from_depth_image(
        depth_image, intrinsic, extrinsic=np.identity(4), depth_scale=1000.0
    )

    # Save the reconstructed point cloud to a file
    o3d.io.write_point_cloud(output_pointcloud_filename, pcd)
    print(f"Reconstructed point cloud saved to {output_pointcloud_filename}")

    # Visualize the point cloud
    o3d.visualization.draw_geometries([pcd])

# Example parameters
depth_filename = "try_decode_input_mult_noclip.png"  # The raw depth image file
output_pointcloud_filename = "reconstructed_pointcloud.ply"

# Intrinsic parameters (adjust based on your setup)
width = 800  # Image width
height = 800  # Image height
fx = 692.82  # Focal length in x direction
fy = 692.82  # Focal length in y direction
cx = width / 2.0  # Principal point in x direction
cy = height / 2.0  # Principal point in y direction

# Call the reconstruction function
reconstruct_from_depth(depth_filename, width, height, fx, fy, cx, cy, output_pointcloud_filename)



Depth image type: uint16
Depth image shape: (800, 800)
Reconstructed point cloud saved to reconstructed_pointcloud.ply
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


: 

In [5]:
def inspect_depth_image(depth_filename):
    # Load the depth image using Open3D
    depth_image = o3d.io.read_image(depth_filename)

    # Convert to a NumPy array for easy inspection
    depth_array = np.asarray(depth_image)

    # Inspect properties
    print(f"Depth image shape: {depth_array.shape}")
    print(f"Min depth value: {depth_array.min()}")
    print(f"Max depth value: {depth_array.max()}")
    print(f"Depth image dtype: {depth_array.dtype}")

    return depth_array

# Example usage
depth_filename = "front_view_depth_o3d.png"
depth_array = inspect_depth_image(depth_filename)

Depth image shape: (800, 800)
Min depth value: 0
Max depth value: 1207
Depth image dtype: uint16


In [2]:
visualizer = Open3DVisualizer()

cuboid_path = "/Users/cmazzoleni/Documents/GitHub/SemesterProjectETH/data/processed/chair_cuboid_json/185bcb9bcec174c9492d9da2668ec34c.json"
cuboid_data = load_cuboid_data(cuboid_path)
visualizer.add_complete_cuboid(cuboid_data)
front_view_direction = [0, 0, -1]  # Front view direction (camera looking towards -z)
output_filename = "front_view_depth_cuboid.png"
depth_o3d_filename = "front_view_depth_o3d_cuboid.png"
    
visualizer.capture_single_view(front_view_direction, depth_o3d_filename, output_filename, save_depth=True)
    



[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
Saved depth image to front_view_depth_cuboid.png


In [9]:
image = Image.open("front_view_depth_o3d.png")

In [10]:
print(image.size)
print(image.mode)   
print(image.format)
print(image.getextrema())

#visualize the image
image.show()

(800, 800)
I;16
PNG
(0, 1207)


: 

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
